# Numeric to word

Converts:
- General number and decimals (Distance, weight, etc)
- Time
    - Hour (enhanced from https://sukhbinder.wordpress.com/2013/12/29/time-in-words-with-python/)
    - Date
    - Year
- Phone number (https://stackabuse.com/validating-and-formatting-phone-numbers-in-python)
- Money (the currency symbols)
    - Find the currency symbols in text:
        - https://www.geeksforgeeks.org/how-to-find-index-of-any-currency-symbols-in-a-given-string/
    - The symbols:
        - https://thefactfile.org/countries-currencies-symbols/ (taken)
        - https://wise.com/gb/blog/world-currency-symbols

Steps:
- Recognizing the number's meaning (4/6 done)
- The calculation itself (5/6 done)
- Corner cases
- Implementation to NL-augmenter

Further improvements:
- Word to numeric
    - https://pypi.org/project/word2number-i18n/
    - https://pypi.org/project/word2number/
- All lang numeric
    - https://pypi.org/project/fastlangid/

In [1]:
import re
import datetime
import phonenumbers
from num2words import num2words
from phonenumbers import carrier, timezone, geocoder

from supplements import symbol_to_currency_name_dict, TimeInWords2

#### Recognizers

In [2]:
def recognized_as_datestring(x):
    # TO DO
    return False

def recognized_as_year(x):
    return bool(re.compile(r'.*([1-3][0-9]{3})').match(x))

def recognized_as_time(x):
    try:
        time.strptime(x, '%H:%M')
        return True
    except ValueError:
        try:
            time.strptime(x, '%H.%M')
            return True
        except ValueError:
            return False

def recognized_as_phone_number(x):
    """
    Accepting phone number format with stripes
    This function also do a check on the number whether it's a valid phone number or not
    (can't accept a random phone number here)
    phone_number = "+62-87986-123456"
    """
    my_number = phonenumbers.parse(x, "ID")
    return phonenumbers.is_valid_number(my_number)

def recognized_as_currency_symbols(x):
    return x in list(symbol_to_currency_name_dict.keys())

def recognized_as_general_numbers(x):
    return x.isnumeric()

#### Transformers

In [3]:
def datestring_to_words(x):
    # TO DO
    return False

def time_to_words(x):
    """
    Currently accepted time formats are:
    - x = "10x30" with x being any non integer hour-minute separator
    - 12h formats (e.g. 24:00 = twelve o'clock, 30:00 = eighteen o'clock)
    
    Possible but not yet catered time formats:
    - Timestamps
    """
    t = TimeInWords2()
    words = t.caltime(x)
    return words

def year_to_words(x):
    words = ''.join(num2words(x, to='year').split(","))
    return words

def phonenum_to_words(x):
    numbers = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
    
    x = re.sub("[^0-9]", "", x)
    
    words = ''
    for number in x:
        words = words + numbers[int(number)] + ' '
    return words

def general_numbers_to_words(x):
    """
    General number and decimals (Distance, weight, etc)
    """
    words = ''.join(num2words(x).split(","))
    return words

#### Implementation

In [4]:
# TO DO: Check whether all the desired capabilities are met

x = # YOUR TEST NUMBER STRINGS HERE #

if recognized_as_datestring(x):
    words = datestring_to_words()
elif recognized_as_time(x):
    words = time_to_words(x)
elif recognized_as_year(x):
    words = year_to_words(x)
elif recognized_as_phone_number(x): 
    words = phonenum_to_words(x)
elif recognized_as_currency_symbols(x):
    words = symbol_to_currency_name_dict[x]
elif recognized_as_general_numbers(x):
    words = general_numbers_to_words(x)
    
# ELSE: Numbers that not in the above stated formats, strings

SyntaxError: invalid syntax (<ipython-input-4-5f05c69c0cf2>, line 3)